## Triangular Arbitrage Scanner

### Libraries and Dependencies

In [8]:
import time
from concurrent.futures import ThreadPoolExecutor

# Has private dependencies
from src.trading_logic import TradingLogicBase
from src.arbitrage.triangular import TriangularArbitrage
from src.market_actor import EchoMarketActor
from src.market_listener import MarketListenerBase, OrderBook, MarketListenerStub
from src.market_listener.huobi import HuobiListener

### Market Listeners

In [9]:
ADA_USDT = MarketListenerStub(market=("ADA", "USDT"))
ADA_ETH = MarketListenerStub(market=("ADA", "ETH"))
ETH_USDT = MarketListenerStub(market=("ETH", "USDT"))

market_listeners = [ADA_USDT, ADA_ETH, ETH_USDT]

In [10]:
def subscribe(market_listeners: list[MarketListenerBase]):
    for market_listener in market_listeners:
        market_listener.subscribe()

    for market_listener in market_listeners:
        while not market_listener.rendered():
            time.sleep(.5) # Busy waiting

In [11]:
subscribe(market_listeners)

In [12]:
mthread_executor = ThreadPoolExecutor(max_workers=3)

### Market Actor
Actors abstract the order execution process

In [13]:
echo = EchoMarketActor(name="demo", transact_fee_rate=0.26/100)

### Trading Logic
Logic determines the price and size of the positions to enter

In [14]:
class CustomTradingLogic (TradingLogicBase):
    # A conservative approach should scan the 2nd entry (index=1) in the orderbook
    scan_entry_num = 0

    def get_size_from_portfolio(self, portfolio_value: float) -> float:
        return 1 # For demo purposes only

    def get_buy_price_from_orderbook(self, orderbook: OrderBook) -> float:
        ask_price, _, _ = orderbook.get_ask(self.scan_entry_num)
        return ask_price

    def get_buy_size_from_orderbook(self, orderbook: OrderBook) -> float:
        _, ask_size, _ = orderbook.get_ask(self.scan_entry_num)
        return ask_size

    def get_sell_price_from_orderbook(self, orderbook: OrderBook) -> float:
        bid_price, _, _ = orderbook.get_bid(self.scan_entry_num)
        return bid_price

    def get_sell_size_from_orderbook(self, orderbook: OrderBook) -> float:
        _, bid_size, _ = orderbook.get_bid(self.scan_entry_num)
        return bid_size

In [15]:
trading_logic = CustomTradingLogic()

### Arbitrage Machine

In [16]:
arb_scanner = TriangularArbitrage(
    origin_curr="USDT",
    trading_logic=trading_logic,
    market_listeners=market_listeners,
    market_actors=[echo, echo, echo]
)

In [17]:
# Sanity checks
print("Forward Path")
for node in arb_scanner.forward_path:
    print(node)

print("\nReverse Path")
for node in arb_scanner.reverse_path:
    print(node)

Forward Path
Long ('ADA', 'USDT')
Short ('ADA', 'ETH')
Short ('ETH', 'USDT')

Reverse Path
Long ('ETH', 'USDT')
Long ('ADA', 'ETH')
Short ('ADA', 'USDT')


### Arbitrage Scanning Test

In [18]:
# Construct an arbitrage opportunity for forward path
ADA_USDT.orderbook.reset()
ADA_ETH.orderbook.reset()
ETH_USDT.orderbook.reset()

ADA_USDT.orderbook.append_ask(0.99, 100)
ADA_ETH.orderbook.append_bid(1, 100)
ETH_USDT.orderbook.append_bid(1, 100)

# No reverse path arbitrage
ADA_USDT.orderbook.append_bid(1, 100)
ADA_ETH.orderbook.append_ask(1, 100)
ETH_USDT.orderbook.append_ask(1, 100)

arb_scanner.scan_and_execute(1)

Buy  1.01 ADA   at 0.99 USDT 
Sell 1.01 ADA   at 1.00 ETH  
Sell 1.00 ETH   at 1.00 USDT 


True

In [19]:
# Construct an arbitrage opportunity for reverse path
ADA_USDT.orderbook.reset()
ADA_ETH.orderbook.reset()
ETH_USDT.orderbook.reset()

# No forward path arbitrage
ADA_USDT.orderbook.append_ask(1, 100)
ADA_ETH.orderbook.append_bid(1, 100)
ETH_USDT.orderbook.append_bid(1, 100)

ADA_USDT.orderbook.append_bid(1.01, 100)
ADA_ETH.orderbook.append_ask(1, 100)
ETH_USDT.orderbook.append_ask(1, 100)

arb_scanner.scan_and_execute(1)

Buy  1.00 ETH   at 1.00 USDT 
Buy  1.00 ADA   at 1.00 ETH  
Sell 1.00 ADA   at 1.01 USDT 


True

### Shutting down Listeners

In [20]:
for market_listener in market_listeners:
    market_listener.close()